In [75]:
import requests
import csv
import arcpy
import os
import io
from io import StringIO
import pandas as pd
from datetime import datetime, timedelta

In [76]:


# Using the datetime tool, we can get today's date and format it as "YYYY-MM-DD"
today_date = datetime.now().strftime("%Y-%m-%d")

In [77]:
#We use the requests tool to get the maximum temperature data as a CSV
#The end of the URL has been adjusted so that it takes in the current date
url = "https://ndawn.ndsu.nodak.edu/table.csv?station=78&station=111&station=98&station=162&station=174&station=142&station=164&station=138&station=161&station=9&station=160&station=159&station=10&station=118&station=56&station=165&station=11&station=12&station=58&station=13&station=84&station=55&station=179&station=7&station=186&station=87&station=14&station=15&station=96&station=191&station=16&station=201&station=137&station=124&station=143&station=17&station=85&station=140&station=134&station=18&station=136&station=65&station=104&station=99&station=192&station=19&station=129&station=20&station=101&station=166&station=178&station=81&station=21&station=97&station=22&station=75&station=184&station=2&station=172&station=139&station=158&station=23&station=157&station=62&station=86&station=24&station=89&station=126&station=167&station=93&station=183&station=90&station=25&station=83&station=107&station=156&station=77&station=26&station=155&station=70&station=127&station=144&station=27&station=173&station=132&station=28&station=195&station=185&station=29&station=30&station=154&station=31&station=187&station=102&station=32&station=119&station=4&station=80&station=33&station=59&station=153&station=105&station=82&station=34&station=198&station=72&station=135&station=35&station=76&station=120&station=141&station=109&station=36&station=79&station=193&station=71&station=37&station=38&station=189&station=39&station=130&station=73&station=188&station=40&station=41&station=54&station=69&station=194&station=145&station=113&station=128&station=42&station=43&station=103&station=171&station=116&station=196&station=88&station=114&station=3&station=163&station=200&station=64&station=115&station=168&station=67&station=175&station=146&station=170&station=197&station=44&station=133&station=106&station=100&station=121&station=45&station=46&station=61&station=66&station=181&station=74&station=60&station=199&station=125&station=176&station=177&station=8&station=180&station=204&station=47&station=122&station=108&station=5&station=152&station=48&station=151&station=147&station=68&station=169&station=49&station=50&station=91&station=182&station=117&station=63&station=150&station=51&station=6&station=52&station=92&station=112&station=131&station=123&station=95&station=53&station=203&station=190&station=57&station=149&station=148&station=202&station=110&variable=ddmxt&year=2023&ttype=daily&quick_pick=30_d&begin_date=2023-11-26&end_date={today_date}"

response = requests.get(url)

In [79]:
#We then read in the CSV's text
#Since there were issues with all the entries on one row ending up in the same cell, the following code breaks them up into columns
data = response.text
lines = data.strip().split('\n')[3:]
result = '\n'.join(lines)

In [81]:
#We then can read the CSV data into a dataframe.
csv_file = StringIO(result)
dataframe = pd.read_csv(csv_file)


In [82]:
#Since the first line after the column headers is units, we can drop that.
dataframe = dataframe.iloc[1:]

In [ ]:
#First we convert the values of the temperature, latitude, and longitude columns to numeric
dataframe['Max Temp'] = pd.to_numeric(dataframe['Max Temp'], errors='coerce')
dataframe['Latitude'] = pd.to_numeric(dataframe['Latitude'], errors='coerce')
dataframe['Longitude'] = pd.to_numeric(dataframe['Longitude'], errors='coerce')

# Then, we group by "Station Name" and calculate the mean for each group and read that into a new dataframe
average_max_temp_df = dataframe.groupby("Station Name").agg({
    "Max Temp":'mean',
    'Latitude':'mean',
    'Longitude':"mean"
}).reset_index()

In [84]:
#Then we perofrm the same procedure to gather the minimum temperature data

min_url = "https://ndawn.ndsu.nodak.edu/table.csv?station=78&station=111&station=98&station=162&station=174&station=142&station=164&station=138&station=161&station=9&station=160&station=159&station=10&station=118&station=56&station=165&station=11&station=12&station=58&station=13&station=84&station=55&station=179&station=7&station=186&station=87&station=14&station=15&station=96&station=191&station=16&station=201&station=137&station=124&station=143&station=17&station=85&station=140&station=134&station=18&station=136&station=65&station=104&station=99&station=192&station=19&station=129&station=20&station=101&station=166&station=178&station=81&station=21&station=97&station=22&station=75&station=184&station=2&station=172&station=139&station=158&station=23&station=157&station=62&station=86&station=24&station=89&station=126&station=167&station=93&station=183&station=90&station=25&station=83&station=107&station=156&station=77&station=26&station=155&station=70&station=127&station=144&station=27&station=173&station=132&station=28&station=195&station=185&station=29&station=30&station=154&station=31&station=187&station=102&station=32&station=119&station=4&station=80&station=33&station=59&station=153&station=105&station=82&station=34&station=198&station=72&station=135&station=35&station=76&station=120&station=141&station=109&station=36&station=79&station=193&station=71&station=37&station=38&station=189&station=39&station=130&station=73&station=188&station=40&station=41&station=54&station=69&station=194&station=145&station=113&station=128&station=42&station=43&station=103&station=171&station=116&station=196&station=88&station=114&station=3&station=163&station=200&station=64&station=115&station=168&station=67&station=175&station=146&station=170&station=197&station=44&station=133&station=106&station=100&station=121&station=45&station=46&station=61&station=66&station=181&station=74&station=60&station=199&station=125&station=176&station=177&station=8&station=180&station=204&station=47&station=122&station=108&station=5&station=152&station=48&station=151&station=147&station=68&station=169&station=49&station=50&station=91&station=182&station=117&station=63&station=150&station=51&station=6&station=52&station=92&station=112&station=131&station=123&station=95&station=53&station=203&station=190&station=57&station=149&station=148&station=202&station=110&variable=ddmnt&year=2023&ttype=daily&quick_pick=30_d&begin_date=2023-11-26&end_date={today_date}"
min_response = requests.get(min_url)

min_data= min_response.text

min_lines = min_data.strip().split('\n')[3:]

min_result = '\n'.join(min_lines)

min_csv_file = StringIO(min_result)

min_dataframe = pd.read_csv(min_csv_file)

min_dataframe = min_dataframe.iloc[1:]

min_dataframe['Min Temp'] = pd.to_numeric(min_dataframe['Min Temp'], errors='coerce')
min_dataframe['Latitude'] = pd.to_numeric(min_dataframe['Latitude'], errors='coerce')
min_dataframe['Longitude'] = pd.to_numeric(min_dataframe['Longitude'], errors='coerce')

average_min_temp_df = min_dataframe.groupby("Station Name").agg({
    "Min Temp":'mean',
    'Latitude':'mean',
    'Longitude':"mean"
}).reset_index()


In [92]:
# We then save the modified dataframes back to the CSV files
average_max_temp_df.to_csv('max_temp.csv', index=False)
average_min_temp_df.to_csv('min_temp.csv', index=False)

In [93]:
#We are going to produe two sets of point features representing NDAWN stations.
#The first will be based off the maximum temperature data and use temperature as the Z field
arcpy.management.XYTableToPoint(
    in_table="max_temp.csv",
    out_feature_class=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5571 Lab3_2\GIS 5571 Lab3_2.gdb\output_modified_XYTableToPoint_max",
    x_field="Longitude",
    y_field="Latitude",
    z_field="Max Temp",
    coordinate_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],VERTCS["WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PARAMETER["Vertical_Shift",0.0],PARAMETER["Direction",1.0],UNIT["Meter",1.0]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision'
)

<Result 'C:\\Users\\conno\\OneDrive\\Documents\\ArcGIS\\Projects\\GIS 5571 Lab3_2\\GIS 5571 Lab3_2.gdb\\output_modified_XYTableToPoint_max'>

In [97]:
#We will create another set for the minimum temperature data.
arcpy.management.XYTableToPoint(
    in_table="min_temp.csv",
    out_feature_class=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5571 Lab3_2\GIS 5571 Lab3_2.gdb\output_modified_XYTableToPoint_min",
    x_field="Longitude",
    y_field="Latitude",
    z_field="Min Temp",
    coordinate_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],VERTCS["WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PARAMETER["Vertical_Shift",0.0],PARAMETER["Direction",1.0],UNIT["Meter",1.0]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision'
)

<Result 'C:\\Users\\conno\\OneDrive\\Documents\\ArcGIS\\Projects\\GIS 5571 Lab3_2\\GIS 5571 Lab3_2.gdb\\output_modified_XYTableToPoint_min'>

In [98]:
#From here, we can create our interpolations. First we will use IDW for the maximum temperature dataset
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5571 Lab3_2\GIS 5571 Lab3_2.gdb"):
    out_raster = arcpy.sa.Idw(
        in_point_features="output_modified_XYTableToPoint_max",
        z_field="Max_Temp",
        cell_size=0.0172421199999999,
        power=2,
        search_radius="VARIABLE 12",
        in_barrier_polyline_features=None
    )
    out_raster.save(r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5571 Lab3_2\GIS 5571 Lab3_2.gdb\Idw_output_m_max")

In [95]:
#Then we do an EBK interpolation for the maximum temperature dataset
arcpy.ga.EmpiricalBayesianKriging(
    in_features="output_modified_XYTableToPoint_max",
    z_field="Max_Temp",
    out_ga_layer=None,
    out_raster=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5571 Lab3_2\GIS 5571 Lab3_2.gdb\EBK_Interpolation_raster_max",
    cell_size=0.0172421199999999,
    transformation_type="NONE",
    max_local_points=100,
    overlap_factor=1,
    number_semivariograms=100,
    search_neighborhood="NBRTYPE=StandardCircular RADIUS=3.1834293129079 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    output_type="PREDICTION",
    quantile_value=0.5,
    threshold_type="EXCEED",
    probability_threshold=None,
    semivariogram_model_type="POWER"
)

<Result ''>

In [96]:
#Finally we create an NNI interpolation for the maximum temperature dataset
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5571 Lab3_2\GIS 5571 Lab3_2.gdb"):
    Natural_outp1_max = arcpy.sa.NaturalNeighbor(
        in_point_features="output_modified_XYTableToPoint_max",
        z_field="Max_Temp",
        cell_size=0.0172421199999999
    )
    Natural_outp1_max.save(r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5571 Lab3_2\GIS 5571 Lab3_2.gdb\Natural_outp1_max")

<Result 'GPI_Output'>

In [99]:
#Once we have those, we can perform the same interpolations for the minimum temperature dataset
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5571 Lab3_2\GIS 5571 Lab3_2.gdb"):
    out_raster = arcpy.sa.Idw(
        in_point_features="output_modified_XYTableToPoint_min",
        z_field="Min_Temp",
        cell_size=0.0172421199999999,
        power=2,
        search_radius="VARIABLE 12",
        in_barrier_polyline_features=None
    )
    out_raster_min.save(r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5571 Lab3_2\GIS 5571 Lab3_2.gdb\Idw_output_min")
    
    arcpy.ga.EmpiricalBayesianKriging(
    in_features="output_modified_XYTableToPoint_min",
    z_field="Min_Temp",
    out_ga_layer=None,
    out_raster=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5571 Lab3_2\GIS 5571 Lab3_2.gdb\EBK_Interpolation_raster_min",
    cell_size=0.0172421199999999,
    transformation_type="NONE",
    max_local_points=100,
    overlap_factor=1,
    number_semivariograms=100,
    search_neighborhood="NBRTYPE=StandardCircular RADIUS=3.1834293129079 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    output_type="PREDICTION",
    quantile_value=0.5,
    threshold_type="EXCEED",
    probability_threshold=None,
    semivariogram_model_type="POWER"
)

    
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5571 Lab3_2\GIS 5571 Lab3_2.gdb"):
    Natural_outp1_min = arcpy.sa.NaturalNeighbor(
        in_point_features="output_modified_XYTableToPoint_min",
        z_field="Min_Temp",
        cell_size=0.0172421199999999
    )
    Natural_outp1_min.save(r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5571 Lab3_2\GIS 5571 Lab3_2.gdb\Natural_outp1_min")

<Result 'GPI_Output'>